In [1]:
import numpy as np

In [2]:
### KNIGHT ALLOWED MOVES (OFFSETS)
allowedOffsetMoves = {(-2,-1),
                      (-1,-2),
                      (+1,-2),
                      (+2,-1),
                      (+2,+1),
                      (+1,+2),
                      (-1,+2),
                      (-2,+1)
}

### SIX BASE STRUCTURED KNIGHT TOURS
## Coords based on chess algebraic notation

####### 6x6 #######
##    # # # # # # 6
##    # # # # # # 5
##    # # # # # # 4
##    + # # # # # 3
##    # # # # # # 2
##    # # # # # # 1
##    1 2 3 4 5 6
##
##    Cell + is (1,3)

path6x6 = {
    (1,1): [(2,3),(3,2)],
    (2,3): [(3,1),(1,1)],
    (3,1): [(1,2),(2,3)],
    (1,2): [(3,3),(3,1)],
    (3,3): [(5,2),(1,2)],
    (5,2): [(6,4),(3,3)],
    (6,4): [(5,6),(5,2)],
    (5,6): [(3,5),(6,4)],
    (3,5): [(1,6),(5,6)],
    (1,6): [(2,4),(3,5)],
    (2,4): [(4,3),(1,6)],
    (4,3): [(5,5),(2,4)],
    (5,5): [(3,6),(4,3)],
    (3,6): [(1,5),(5,5)],
    (1,5): [(3,4),(3,6)],
    (3,4): [(2,6),(1,5)],
    (2,6): [(1,4),(3,4)],
    (1,4): [(2,2),(2,6)],
    (2,2): [(4,1),(1,4)],
    (4,1): [(6,2),(2,2)],
    (6,2): [(5,4),(4,1)],
    (5,4): [(6,6),(6,2)],
    (6,6): [(4,5),(5,4)],
    (4,5): [(5,3),(6,6)],
    (5,3): [(6,1),(4,5)],
    (6,1): [(4,2),(5,3)],
    (4,2): [(2,1),(6,1)],
    (2,1): [(1,3),(4,2)],
    (1,3): [(2,5),(2,1)],
    (2,5): [(4,6),(1,3)],
    (4,6): [(6,5),(2,5)],
    (6,5): [(4,4),(4,6)],
    (4,4): [(6,3),(6,5)],
    (6,3): [(5,1),(4,4)],
    (5,1): [(3,2),(6,3)],
    (3,2): [(1,1),(5,1)]
}

In [3]:
class Chessboard:
    
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        self.knightAdjacencyList = {}
        self.knightTour = {}
        
    def SetAdjacencyList(self, knightAdjacencyList):
        self.knightAdjacencyList = knightAdjacencyList.copy()
    
    def GetAdjacencyList(self):
        return self.knightAdjacencyList.copy()
    
    def SetTour(self, tour):
        self.knightTour = tour.copy()
    
    def GetTour(self):
        return self.knightTour.copy()
    
    def GetRows(self):
        return self.rows
    
    def GetColumns(self):
        return self.columns
    
    def FindAdjacencyList(self):
        if((self.GetRows() == 6) and (self.GetColumns() == 6)):
            self.SetAdjacencyList(path6x6)
            return
        
        nRows = self.GetRows()
        nColumns = self.GetColumns()
        
        topLeftBoard = Chessboard(nRows/2, nColumns/2)
        topRightBoard = Chessboard(nRows/2, nColumns/2)
        bottomLeftBoard = Chessboard(nRows/2, nColumns/2)
        bottomRightBoard = Chessboard(nRows/2, nColumns/2)
        
        topLeftBoard.FindAdjacencyList()
        topRightBoard.FindAdjacencyList()
        bottomLeftBoard.FindAdjacencyList()
        bottomRightBoard.FindAdjacencyList()
        
        ## Build new adjacency list from bottom left, makes sense for coordinates
        bottomLeftADL = bottomLeftBoard.GetAdjacencyList()
        bottomRightADL = bottomRightBoard.GetAdjacencyList()
        topLeftADL = topLeftBoard.GetAdjacencyList()
        topRightADL = topRightBoard.GetAdjacencyList()
        
        newBottomLeftADL = {}
        newBottomRightADL = {}
        newTopLeftADL = {}
        newTopRightADL = {}
        
        for position in bottomLeftADL:
            newBottomLeftADL[position] = []
            for adjacentSquare in bottomLeftADL[position]:
                newBottomLeftADL[position].append(adjacentSquare)        
        
        for position in bottomRightADL:
            ## Add columns of bottom-left to column indexes of bottom-right
            newPosition = tuple(map(lambda i, j: int(i + j), position, (bottomLeftBoard.GetColumns(), 0)))
            newBottomRightADL[newPosition] = []
            for adjacentSquare in bottomRightADL[position]:
                ## Add columns of bottom-left to column indexes of bottom-right
                newAdjacentSquare = tuple(map(lambda i, j: int(i + j), adjacentSquare, (bottomLeftBoard.GetColumns(), 0)))
                newBottomRightADL[newPosition].append(newAdjacentSquare)
        
        for position in topLeftADL:
            ## Add rows of bottom-left to row indexes of top-left
            newPosition = tuple(map(lambda i, j: int(i + j), position, (0, bottomLeftBoard.GetRows())))
            newTopLeftADL[newPosition] = []
            for adjacentSquare in topLeftADL[position]:
                ## Add rows of bottom-left to row indexes of top-left
                newAdjacentSquare = tuple(map(lambda i, j: int(i + j), adjacentSquare, (0, bottomLeftBoard.GetRows())))
                newTopLeftADL[newPosition].append(newAdjacentSquare)

        for position in topRightADL:
            ## Add rows & columns to top-right
            newPosition = tuple(map(lambda i, j: int(i + j), position, (bottomLeftBoard.GetColumns(), bottomLeftBoard.GetRows())))
            newTopRightADL[newPosition] = []
            for adjacentSquare in topRightADL[position]:
                ## Add rows & columns to top-right
                newAdjacentSquare = tuple(map(lambda i, j: int(i + j), adjacentSquare, (bottomLeftBoard.GetColumns(), bottomLeftBoard.GetRows())))
                newTopRightADL[newPosition].append(newAdjacentSquare)
                
        ### Fix the edges
        ## Relevant squares
        #A1 = (4,7)
        #A2 = (6,8)
        #B1 = (7,7)
        #B2 = (8,9)
        #C1 = (7,5)
        #C2 = (9,6)
        #D1 = (5,4)
        #D2 = (6,6)
        A1 = (bottomLeftBoard.GetColumns()-2, bottomLeftBoard.GetRows()+1)
        A2 = (bottomLeftBoard.GetColumns(),   bottomLeftBoard.GetRows()+2)
        B1 = (bottomLeftBoard.GetColumns()+1, bottomLeftBoard.GetRows()+1)
        B2 = (bottomLeftBoard.GetColumns()+2, bottomLeftBoard.GetRows()+3)
        C1 = (bottomLeftBoard.GetColumns()+1, bottomLeftBoard.GetRows()-1)
        C2 = (bottomLeftBoard.GetColumns()+3, bottomLeftBoard.GetRows()  )
        D1 = (bottomLeftBoard.GetColumns()-1, bottomLeftBoard.GetRows()-2)
        D2 = (bottomLeftBoard.GetColumns(),   bottomLeftBoard.GetRows()  )
        ## Pop edge A
        newTopLeftADL[A1].remove(A2)
        newTopLeftADL[A2].remove(A1)
        ## Pop edge B
        newTopRightADL[B1].remove(B2)
        newTopRightADL[B2].remove(B1)
        ## Pop edge C
        newBottomRightADL[C1].remove(C2)
        newBottomRightADL[C2].remove(C1)
        ## Pop edge D
        newBottomLeftADL[D1].remove(D2)
        newBottomLeftADL[D2].remove(D1)
        ## Add edge E
        newBottomLeftADL[D2].append(A1)
        newTopLeftADL[A1].append(D2)
        ## Add edge F
        newTopLeftADL[A2].append(B2)
        newTopRightADL[B2].append(A2)
        ## Add edge G
        newTopRightADL[B1].append(C2)
        newBottomRightADL[C2].append(B1)
        ## Add edge H
        newBottomRightADL[C1].append(D1)
        newBottomLeftADL[D1].append(C1)
        
        newCompleteADL = {**newBottomLeftADL, **newBottomRightADL, **newTopLeftADL, **newTopRightADL}
        self.SetAdjacencyList(newCompleteADL)
        return
    
    def FindTour(self):
        startingPosition = (1,1)
        currentPosition = startingPosition
        adjacencyList = self.GetAdjacencyList()
        visitedSquares = {startingPosition: True}
        tour = {}

        while True:
            exhaustedList = False

            for square in adjacencyList[currentPosition]:
                if square not in visitedSquares:
                    visitedSquares[square] = True
                    tour[currentPosition] = square
                    currentPosition = square
                    break
                exhaustedList = True

            if exhaustedList:
                if tuple(map(lambda i,j: i - j, currentPosition, startingPosition)) in allowedOffsetMoves:
                    tour[currentPosition] = startingPosition
                    break
        
        if len(visitedSquares) != (self.GetRows() * self.GetColumns()):
            print("Ah-le-le?! The tour ends after", len(visitedSquares), "squares instead of", self.GetRows() * self.GetColumns() ,".")
        
        self.SetTour(tour)
    
    def PrintTour(self):
        nextStep = self.GetTour()
        startingPosition = (1,1)
        currentPosition = nextStep[startingPosition]
        
        tourMatrix = np.zeros((self.GetRows(),self.GetColumns()),int)
        
        visitedPositions = {startingPosition: 1}
        tourMatrix[self.GetRows()-startingPosition[1],startingPosition[0]-1] = 1
        positionCounter = 1

        while currentPosition !=  startingPosition:
            if currentPosition in visitedPositions:
                print("ERROR: square", currentPosition, "is repeating.")
                print("Visited at steps", visitedPositions[currentPosition], "and", positionCounter,".")
                break
            
            positionCounter += 1
            visitedPositions[currentPosition] = positionCounter
            tourMatrix[int(self.GetRows()-currentPosition[1]),int(currentPosition[0]-1)] = positionCounter
            currentPosition = nextStep[currentPosition]
        
        if len(visitedPositions) != self.GetRows() * self.GetColumns():
            print("ERROR: ", len(visitedPositions), "visited squares, but", self.GetRows()*self.GetColumns(), "squares on the board.")
        
        print(tourMatrix)

    def TourIsStructured(self):
        adjacencyList = self.GetAdjacencyList()
        columns = self.GetColumns()
        rows = self.GetRows()
        
        condBL1 = ((1,3) in adjacencyList[(2,1)])
        condBL2 = ((1,2) in adjacencyList[(3,1)])
        condTL1 = ((1,rows-1) in adjacencyList[(3,rows)])
        condTL2 = ((1,rows-2) in adjacencyList[(2,rows)])
        condTR1 = ((columns-2,rows) in adjacencyList[(columns,rows-1)])
        condTR2 = ((columns-1,rows) in adjacencyList[(columns,rows-2)])
        condBR1 = ((columns-2,1) in adjacencyList[(columns,2)])
        condBR2 = ((columns-1,1) in adjacencyList[(columns,3)])
        
        return (condBL1 and condBL2 and condTL1 and condTL2 and condTR1 and condTR2 and condBR1 and condBR2)
    
    def TourIsComplete(self):
        return len(self.GetTour()) == self.GetColumns() * self.GetRows()
    
    def TourIsClosed(self):
        startingPosition = (1,1)
        currentPosition = startingPosition
        tourNextStep = self.GetTour()
        
        for i in range(self.GetColumns() * self.GetRows()):
            currentPosition = tourNextStep[currentPosition]
        
        return currentPosition == startingPosition
    
    def TourIsNonRepeating(self):
        startingPosition = (1,1)
        currentPosition = startingPosition
        tourNextStep = self.GetTour()
        
        ## Skip first position because it's also the last
        visitedPositions = {}
        isNonRepeating = True
        
        for i in range(self.GetColumns() * self.GetRows()):
            currentPosition = tourNextStep[currentPosition]
            if currentPosition in visitedPositions:
                isNonRepeating = False
                break
                
        return isNonRepeating
    
    def DebugTour(self):
        print("Complete:", self.TourIsComplete())
        print("Closed:", self.TourIsClosed())
        print("Structured:", self.TourIsStructured())
        print("Non-repeating:", self.TourIsNonRepeating())

In [4]:
board = Chessboard(12,12)
print(board.GetRows())
print(board.GetColumns())

12
12


In [5]:
print("Building knight adjacency list...")
board.FindAdjacencyList()
print("Building knight tour...")
board.FindTour()
#print((board.GetTour()))
print()
board.PrintTour()

Building knight adjacency list...
Building knight tour...

[[120 127 124 105 118  97  86  79  82  65  88  73]
 [125 104 119  98 123 106  81  66  87  72  83  64]
 [128 121 126 107  96 117  78  85  80  63  74  89]
 [103 112 115 122  99 108  67  94  91  84  71  62]
 [114 129 110 101 116  95  92  77  60  69  90  75]
 [111 102 113 130 109 100  59  68  93  76  61  70]
 [ 10  17  14 139   8 131  54  25  58  39  52  31]
 [ 15 138   9 132  13 140  23  38  53  32  57  40]
 [ 18  11  16 141  22   7  26  55  24  41  30  51]
 [137   2   5  12 133 142  37  46  49  56  33  42]
 [  4  19 144 135   6  21  48  27  44  35  50  29]
 [  1 136   3  20 143 134  45  36  47  28  43  34]]


In [6]:
print("ADL length:", len(board.GetAdjacencyList()))
print("Tot. squares:", board.GetColumns() * board.GetRows())

ADL length: 144
Tot. squares: 144


In [7]:
board.DebugTour()

Complete: True
Closed: True
Structured: True
Non-repeating: True


In [8]:
## To do:
## 1. Add the remaining known tours
## 2. Extend to non-6x6 (should be pretty straightforward)